In [1]:
import evennia
evennia._init()

### Mob Difficulty Calculations for HP, MP, SP

main code used to experiment as well as act as a guideline to calculate
all vital attributes

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from evennia.contrib.dice import roll_dice

class ndict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for k,v in self.items():
            setattr(self, k, v)
    def __setattr__(self, k, v):
        super().__setattr__(k,v)
        self[k] = v
    
    def __getattr__(self, k):
        return self[k]

class CalcVitals:
    def __init__(self, **kwargs):
        self.value = None
        for k,v in kwargs.items():
            setattr(self, k,v)
                    
    def calc(self):
        raise NotImplementedError()
        
    @property
    def base(self):
        return (self.base_mult*np.log2(self.olvl))*self.olvl
        
    def __repr__(self):
        return str(self.calc())

class Hp(CalcVitals):
    def calc(self):
        val =  int((self.base *np.log(self.end)) *self.diff_mod)
        return max(val,3)
class Wp(CalcVitals):
    def calc(self):
        val= int((self.base *np.log((self.int+self.wp)/2))*self.diff_mod)
        return max(val,3)
class Sp(CalcVitals):
    def calc(self):
        val= int((self.base *np.log((self.end+self.agi)/2))*self.diff_mod)
        return max(val,3)
    
stat_names = ['str', 'end','agi', 'int', 'wp','prc', 'prs', 'lck']
diff_mod = 2
diffs = ('wimpy', 'easy', 'medium', 'hard','insane')

m = ndict(
olvl = 1,
base_mult = 1.5,
base_stat = 8,
num_dice = 2,
size_dice = 5,
diff_mod = -1
)


dice = np.vectorize(lambda x: x + roll_dice(m.num_dice, m.size_dice))
stats = dice(np.full((1,8), fill_value=m.base_stat, dtype=np.float64))
stats = np.tile(stats, (250,1))

for idx, diff in enumerate(diffs):
    idx -= 1
#     stats = np.full((250,8), fill_value=m.base_stat, dtype=np.float64)

#     base = dice(stats)
    base = stats

    results = []
    print(f"{diff.capitalize()} {diff_mod ** idx}x")
    
    for level, data in enumerate(base):
        if (level+1) % 50 == 0 or (level) == 0:
            n = ndict({**m, **ndict(zip(stat_names, data))})
            n.olvl = level+1
            n.diff_mod = (diff_mod**idx)
            results.append([n.olvl, Hp(**n).calc(), Wp(**n).calc(), Sp(**n).calc()]+data.tolist())
            
        

    from tabulate import tabulate
    print(tabulate(results,headers=['hp','wp', 'sp']+stat_names)+'\n')
#     break


Wimpy 0.5x
       hp    wp    sp    str    end    agi    int    wp    prc    prs    lck
---  ----  ----  ----  -----  -----  -----  -----  ----  -----  -----  -----
  1     3     3     3     15     12     11     12    11     11     15     11
 50   525   516   516     15     12     11     12    11     11     15     11
100  1238  1216  1216     15     12     11     12    11     11     15     11
150  2020  1986  1986     15     12     11     12    11     11     15     11
200  2849  2800  2800     15     12     11     12    11     11     15     11
250  3711  3647  3647     15     12     11     12    11     11     15     11

Easy 1x
       hp    wp    sp    str    end    agi    int    wp    prc    prs    lck
---  ----  ----  ----  -----  -----  -----  -----  ----  -----  -----  -----
  1     3     3     3     15     12     11     12    11     11     15     11
 50  1051  1033  1033     15     12     11     12    11     11     15     11
100  2476  2433  2433     15     12     11     12    11 

### Experimentation with calcing dam_roll statistics based on level

The base assumption.

If two equal level mobs are fighting with same hp, wp, sp and doing nothing special. Their should be a 50/50 chance the other wins in 5 rounds. From this baseline, we can calculate the potential dam_roll and hit based on level

Each tier of difficulty of the above situation falls in favor on the mob by a 15% difference.. 

Chance of Winning for Player

| Difficulty | Player |  Mob
|------------|--------|-----
|  wimpy     |  90    | 10
|  easy      |  50    | 50
|  medium    |  40    | 60
|  hard      |  20    | 80
|  insane    |  10    | 90
|  chaotic   |   5    | 95
